## 1. Bài toán đặt ra

### Tên bài toán
 Dự đoán số điểm tổng kết của khách hàng đánh giá cho hãng hàng không British Airways
 
### Giới thiệu chung 
- Trong học máy, **học có giám sát** là một nhóm các thuật toán phổ biến trong lĩnh vực này và một trong những vấn đề quan trọng của học có giám sát là hồi quy(regression). Hồi quy là các bài toán liên quan đến việc dự đoán đầu ra có giá trị liên tục (predicting continous valued output).
- Và trong bài toán mà nhóm đề ra thì từ những cột thuộc tính đầu vào như loại ghế, loại khách, đánh giá về chỗ ngồi, chất lượng phục vụ của nhân viên, thức ăn và đồ uống,.... Nhóm tiến hành dự đoán cột Overall_rating là số điểm tổng kết của khách hàng đánh giá bằng thuật toán hồi quy tuyến tính (linear regression).

## 2. Thêm thư viện

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV

#import for decision tree
from sklearn.tree import DecisionTreeClassifier

# import for random forest
from sklearn.ensemble import RandomForestClassifier
# import for GradientBoosting
from sklearn.ensemble import GradientBoostingClassifier


## 3. Chuẩn bị dữ liệu

### Đọc dữ liệu đã qua tiền xử lí

In [2]:
data = pd.read_csv('../data/cleaned_data.csv')

### Loại những thuộc tính không có ý nghĩa cho việc mô hình hóa
- Như đã quan sát ở phần khám phá dữ liệu: Cột `Date_published`, `Name`, `Route`, `Date_flown` chứa các giá trị quá riêng biệt, không có ý nghĩa cho việc trực quan hay phân tích do đó các cột này sẽ không được lựa chọn làm thuộc tính đầu vào.
- Cột `Recommended` hầu như dựa vào số điểm của cột `Overall_rating` để đề xuất nên cũng không có giá trị cho bài toán
- Các cột của bộ dữ liệu được giữ lại bao gồm: `Verified_review`, `Type_of_traveller`, `Seat_type`, `Seat_comfort`, `Cabin_staff_service`, `Food_and_beverages`, `Inflight_entertainments`, `Ground_service	`, `Value_for_money`, `Overall_rating`

In [3]:
data = data.drop(columns=['Date_published', 'Name', 'Route', 'Date_flown', 'Recommended'])
data.head()

,Verified_review,Type_of_traveller,Seat_type,Seat_comfort,Cabin_staff_service,Food_and_beverages,Inflight_entertainments,Ground_service,Value_for_money,Overall_rating
0,Trip Verified,Family Leisure,Business Class,4,5,5,3,5,5,10
1,Trip Verified,Solo Leisure,Economy Class,1,1,3,3,1,1,1
2,Not Verified,Family Leisure,Premium Economy,3,3,3,3,1,1,1
3,Trip Verified,Solo Leisure,Business Class,3,4,1,3,3,5,6
4,Not Verified,Couple Leisure,Business Class,4,5,1,3,5,5,6


### Chuyển đổi các cột không phải dạng số về dạng số 
Chuyển các cột `Verified_review`, `Type_of_traveller`, `Seat_type` về dạng số để làm đầu vào cho mô hình học máy, tuy nhiên các cột có dạng số này vẫn mang ý nghĩa phân loại.

In [4]:
enc = OrdinalEncoder()
data[['Verified_review', 'Type_of_traveller', 'Seat_type']]=enc.fit_transform(np.array(data[['Verified_review', 'Type_of_traveller',
                                                            'Seat_type']]))
data.head()

,Verified_review,Type_of_traveller,Seat_type,Seat_comfort,Cabin_staff_service,Food_and_beverages,Inflight_entertainments,Ground_service,Value_for_money,Overall_rating
0,1.0,2.0,0.0,4,5,5,3,5,5,10
1,1.0,3.0,1.0,1,1,3,3,1,1,1
2,0.0,2.0,3.0,3,3,3,3,1,1,1
3,1.0,3.0,0.0,3,4,1,3,3,5,6
4,0.0,1.0,0.0,4,5,1,3,5,5,6


## 4. Xây dựng mô hình học máy

- Sau khi chuyển các cột không phải số về số, ta sẽ quan sát và phân tích dữ liệu để quyết định mô hình phù hợp cho bài toán.
- Có thể thấy, cột overall_rating(output) có kiểu dữ liệu là số nguyên từ 1-10, đây là kiểu dữ liệu rời rạc.
- Những cột đặc trưng khác (input) cũng tương tự như vậy.
- Do đó ta sẽ sử dụng các thuật toán phân lớp để giải quyết bài toán này.
- Phần dưới sẽ thực hiện 3 mô hình là : Decision Tree, Random Forest và Gradient Boosting.
- Ngoài ra còn thực hiện thêm Grid_search để chọn ra các siêu tham số tốt nhất của mỗi mô hình dựa vào accuracy.

#### Chuẩn bị dữ liệu

In [5]:
X = data.drop('Overall_rating',axis=1)
y = data['Overall_rating']
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### a) Decision Tree:

- Cây quyết định là một thuật toán học có giám sát, có thể được sử dụng để giải quyết cả vấn đề phân loại và hồi quy.
- Nó có thể giải quyết các vấn đề cho cả dữ liệu phân loại và số
- Cây quyết định xây dựng một cấu trúc giống như cây, trong đó mỗi nút bên trong đại diện cho “kiểm tra” cho một thuộc tính, mỗi nhánh đại diện cho kết quả của thử nghiệm và mỗi nút lá đại diện cho quyết định hoặc kết quả cuối cùng.
- Siêu tham số quan trọng nhất: max_depth (độ sâu của cây)

#### Chọn siêu tham số và xây dựng mô hình

In [6]:
# Định nghĩa các giá trị của siêu tham số muốn thử nghiệm
# ở đây ta để tập siêu tham số nhỏ, để máy chạy nhanh hơn.
param_grid = {
    'max_depth': [3,4],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2, 4]
}

# Tạo mô hình Decision Tree Classifier
model = DecisionTreeClassifier()

# Sử dụng GridSearchCV để tìm kiếm siêu tham số tốt nhất
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# In ra siêu tham số tốt nhất
best_params = grid_search.best_params_
print(f'Best Parameters: {best_params}')

best_model = grid_search.best_estimator_
best_model

Best Parameters: {'max_depth': 4, 'min_samples_leaf': 1, 'min_samples_split': 2}


DecisionTreeClassifier(max_depth=4)

#### Dự đoán trên tập kiểm tra

In [7]:
y_pred = best_model.predict(X_test)

dic={'Predictions by Sklearn': list(y_pred), 'Real Values':list(y_test)}
comparison_df=pd.DataFrame(dic)
print(comparison_df)

     Predictions by Sklearn  Real Values
0                         1            1
1                         3            2
2                         3            4
3                        10           10
4                         1            1
..                      ...          ...
595                       1            1
596                       1            2
597                       1            1
598                       6            4
599                       1            2

[600 rows x 2 columns]


### Tính hiệu suất

In [8]:
# Đánh giá hiệu suất của mô hình
tree_accuracy = accuracy_score(y_test, y_pred)
tree_classification_rep = classification_report(y_test, y_pred)
print('Classification Report:\n', tree_classification_rep)
print(f'Decision Tree Accuracy on Test Set: {tree_accuracy}')

Classification Report:
               precision    recall  f1-score   support

           1       0.62      0.92      0.74       159
           2       0.40      0.03      0.05        75
           3       0.21      0.32      0.25        53
           4       0.00      0.00      0.00        40
           5       0.00      0.00      0.00        42
           6       0.25      0.57      0.35        47
           7       0.00      0.00      0.00        41
           8       0.40      0.69      0.51        67
           9       0.30      0.22      0.25        36
          10       0.55      0.45      0.49        40

    accuracy                           0.44       600
   macro avg       0.27      0.32      0.27       600
weighted avg       0.35      0.44      0.36       600

Decision Tree Accuracy on Test Set: 0.44


/home/test/miniconda3/envs/min_ds-env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/test/miniconda3/envs/min_ds-env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/test/miniconda3/envs/min_ds-env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

### b) Random Forest

- Mô hình rừng cây được huấn luyện dựa trên sự phối hợp giữa luật kết hợp (ensembling) và quá trình lấy mẫu tái lặp (boostrapping).
- Cụ thể thuật toán này tạo ra nhiều cây quyết định mà mỗi cây quyết định được huấn luyện dựa trên nhiều mẫu con khác nhau.
- Và kết quả dự báo là bầu cử (voting) từ toàn bộ những cây quyết định.
- Như vậy một kết quả dự báo được tổng hợp từ nhiều mô hình nên kết quả của chúng sẽ không bị chệch. Đồng thời kết hợp kết quả dự báo từ nhiều mô hình sẽ có phương sai nhỏ hơn so với chỉ một mô hình.
- Siêu tham số quan trọng: n_estimators (số lượng cây)

#### Chọn siêu tham số và xây dựng mô hình

In [9]:
# Định nghĩa các giá trị của siêu tham số muốn thử nghiệm
# ở đây ta để tập siêu tham số nhỏ, để máy chạy nhanh hơn.
param_grid = {
    'n_estimators': [10,15,20],
    'max_depth': [3,4],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2, 4]
}

# Tạo mô hình Random Forest Classifier
model = RandomForestClassifier()

# Sử dụng GridSearchCV để tìm kiếm siêu tham số tốt nhất
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# In ra siêu tham số tốt nhất
best_params = grid_search.best_params_
print(f'Best Parameters: {best_params}')

# Lấy mô hình với siêu tham số tốt nhất
best_model = grid_search.best_estimator_
best_model

Best Parameters: {'max_depth': 4, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 15}


RandomForestClassifier(max_depth=4, min_samples_leaf=2, n_estimators=15)

#### Dự đoán trên tập kiểm tra

In [10]:
y_pred = best_model.predict(X_test)

dic={'Predictions by Sklearn': list(y_pred), 'Real Values':list(y_test)}
comparison_df=pd.DataFrame(dic)
print(comparison_df)

     Predictions by Sklearn  Real Values
0                         1            1
1                         3            2
2                         3            4
3                        10           10
4                         1            1
..                      ...          ...
595                       1            1
596                       1            2
597                       1            1
598                       1            4
599                       1            2

[600 rows x 2 columns]


In [11]:
# Đánh giá hiệu suất của mô hình
forest_accuracy = accuracy_score(y_test, y_pred)
forest_classification_rep = classification_report(y_test, y_pred)

print(f'Decision Tree Accuracy on Test Set: {forest_accuracy}')

Decision Tree Accuracy on Test Set: 0.45166666666666666


/home/test/miniconda3/envs/min_ds-env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/test/miniconda3/envs/min_ds-env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/test/miniconda3/envs/min_ds-env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

### c) Gradient Boosting

- Mô hình Gradient Boosting là một phương pháp học máy thuộc dạng "Ensemble Learning" như RandomForest, nhưng thay vì xây dựng nhiều mô hình độc lập nhau như Bagging, Gradient Boosting xây dựng các mô hình theo cách tuần tự
- Nó dựa vào trực giác rằng mô hình tiếp theo tốt nhất có thể, khi kết hợp với các mô hình trước đó, sẽ giảm thiểu sai số dự đoán tổng thể.
- Ý tưởng chính là đặt ra kết quả mục tiêu cho mô hình tiếp theo này để giảm thiểu sai sót.
- Siêu tham số quan trọng: n-estimators, learning_rate.

#### Chọn siêu tham số và xây dựng mô hình

In [12]:
#Định nghĩa các giá trị của siêu tham số muốn thử nghiệm
# ở đây ta để tập siêu tham số nhỏ, để máy chạy nhanh hơn.
param_grid = {
    'n_estimators': [10,15,20],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 4],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2, 4]
}

# Tạo mô hình Gradient Boosting Classifier
gb_model = GradientBoostingClassifier()

# Sử dụng GridSearchCV để tìm kiếm siêu tham số tốt nhất
grid_search = GridSearchCV(gb_model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# In ra siêu tham số tốt nhất
best_params = grid_search.best_params_
print(f'Best Parameters: {best_params}')

# Lấy mô hình với siêu tham số tốt nhất
best_gb_model = grid_search.best_estimator_
best_gb_model

Best Parameters: {'learning_rate': 0.1, 'max_depth': 3, 'min_samples_leaf': 4, 'min_samples_split': 5, 'n_estimators': 20}


GradientBoostingClassifier(min_samples_leaf=4, min_samples_split=5,
                           n_estimators=20)

#### Dự đoán trên tập kiểm tra

In [13]:
# Dự đoán trên tập kiểm tra
y_pred = best_gb_model.predict(X_test)

dic={'Predictions by Sklearn': list(y_pred), 'Real Values':list(y_test)}
comparison_df=pd.DataFrame(dic)
print(comparison_df)

     Predictions by Sklearn  Real Values
0                         1            1
1                         2            2
2                         2            4
3                        10           10
4                         1            1
..                      ...          ...
595                       1            1
596                       1            2
597                       1            1
598                       1            4
599                       1            2

[600 rows x 2 columns]


### Tính hiệu suất

In [14]:
# Đánh giá hiệu suất của mô hình
gb_accuracy = accuracy_score(y_test, y_pred)
gb_classification_rep = classification_report(y_test, y_pred)

print(f'Accuracy on Test Set: {gb_accuracy}')

Accuracy on Test Set: 0.46


## 4. Đánh giá mô hình

#### Ta sẽ dựa vào bảng Classification Report để đánh giá các mô hình này.
Giải thích về các chỉ số có trong report
- $Precision (Độ Chính Xác)$: đo lường tỷ lệ của các dự đoán dương tính thực sự trong số các dự đoán dương tính của mô hình. Nếu bạn quan trọng hóa việc tránh dự đoán sai là dương tính (False Positives), bạn có thể chọn mô hình có precision cao hơn.
- $Recall (Độ Phủ)$: đo lường tỷ lệ của các dự đoán dương tính thực sự trong số tất cả các điểm dương tính thực sự. Nếu bạn quan trọng hóa việc tránh bỏ sót các điểm dương tính (False Negatives), bạn có thể chọn mô hình có recall cao hơn.
- $F1-Score$: là sự kết hợp giữa Precision và Recall. Nó là một số đo tổng hợp giữa việc giảm False Positives và False Negatives. Nếu cả Precision và Recall đều quan trọng trong bài toán của bạn, bạn có thể sử dụng F1-Score để đánh giá tổng thể.
- $Support$: là số lượng mẫu thực sự thuộc mỗi lớp. Điều này có thể giúp bạn hiểu về phân phối của các lớp trong dữ liệu.
- $Accuracy$: Trong bảng classification report có thể cung cấp giá trị accuracy tổng thể của mô hình. Tuy nhiên, accuracy không phải lúc nào cũng là một độ đo tốt, đặc biệt là khi dữ liệu không cân bằng.
- $Macro-Averaging$: Nếu bạn đang làm việc với dữ liệu không cân bằng, bạn có thể quan tâm đến các phương pháp macro-averaging và micro-averaging để tổng hợp các chỉ số từ các lớp khác nhau.

### a) Đánh giá mô hình Tree decision

In [15]:
print('Tree Classification Report:\n', tree_classification_rep)

Tree Classification Report:
               precision    recall  f1-score   support

           1       0.62      0.92      0.74       159
           2       0.40      0.03      0.05        75
           3       0.21      0.32      0.25        53
           4       0.00      0.00      0.00        40
           5       0.00      0.00      0.00        42
           6       0.25      0.57      0.35        47
           7       0.00      0.00      0.00        41
           8       0.40      0.69      0.51        67
           9       0.30      0.22      0.25        36
          10       0.55      0.45      0.49        40

    accuracy                           0.44       600
   macro avg       0.27      0.32      0.27       600
weighted avg       0.35      0.44      0.36       600



### b) Đánh giá mô hình Random Forest

In [16]:
print('Classification Report:\n', forest_classification_rep)

Classification Report:
               precision    recall  f1-score   support

           1       0.61      0.96      0.75       159
           2       0.00      0.00      0.00        75
           3       0.22      0.36      0.27        53
           4       0.17      0.12      0.14        40
           5       0.30      0.07      0.12        42
           6       1.00      0.02      0.04        47
           7       0.21      0.24      0.23        41
           8       0.44      0.70      0.54        67
           9       0.27      0.08      0.13        36
          10       0.53      0.78      0.63        40

    accuracy                           0.45       600
   macro avg       0.37      0.33      0.28       600
weighted avg       0.41      0.45      0.37       600



### c) Đánh giá mô hình Gradient Boosting

In [17]:
print('Classification Report:\n', gb_classification_rep)

Classification Report:
               precision    recall  f1-score   support

           1       0.63      0.92      0.75       159
           2       0.20      0.08      0.11        75
           3       0.31      0.38      0.34        53
           4       0.13      0.10      0.11        40
           5       0.15      0.10      0.12        42
           6       0.31      0.19      0.24        47
           7       0.33      0.32      0.32        41
           8       0.48      0.54      0.51        67
           9       0.33      0.25      0.29        36
          10       0.63      0.72      0.67        40

    accuracy                           0.46       600
   macro avg       0.35      0.36      0.35       600
weighted avg       0.40      0.46      0.42       600



**Nhận xét:**
- Dựa vào accuracy, ta có thể thấy: Gradient Boosting > Random Forest > Tree Decision. ( 0.46 > 0.455 (có in ra ở phần 3) > 0.44)
- Nhìn chung thì accuracy khá thấp. (dưới 50%) (do bộ dữ liệu còn khá ít, cũng như phân bố của các lớp không đồng đều)
- Do đó chưa thể dùng accuracy để đánh giá mô hình nào tốt hơn.
- Vì vậy ta sẽ sử dụng Macro avg ở precision để đánh giá các mô hình.

**Giải thích thêm về $Macro-Averaging$:**
- Macro-averaging tính toán các chỉ số cho mỗi lớp riêng rẽ và sau đó tính trung bình của chúng.
- Mỗi lớp có cùng một trọng số trong tính toán, không phụ thuộc vào số lượng mẫu trong mỗi lớp.
- Macro-averaging trung bình các chỉ số từ mỗi lớp, giúp đánh giá hiệu suất trung bình trên tất cả các lớp mà không quan trọng số lượng mẫu trong mỗi lớp.

**Nhận xét:**
- Dựa vào Micro avg ở cột precision ta có thể thấy: Random Forest > Gradient Boosting > Tree Decision. ( 0.39 > 0.35 > 0.37)
- Do đó tạm thời, với bộ dữ liệu trên, có thể thấy Random Forest là tối ưu nhất.
- Điều đó là dễ hiểu ( do rừng thì đương nhiên tốt hơn một cây).